In [9]:
pip install PIL

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [10]:
import cv2
import pandas as pd
import numpy as np
from PIL import Image

In [15]:
def get_limits(color):
    # Convert RGB to HSV
    color_hsv = cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_RGB2HSV)[0][0]
    
    # Set a tolerance (range) for the color
    tolerance = 30
    lowerlimit = np.array([color_hsv[0] - tolerance, 50, 50])  # Hue, Saturation, Value
    upperlimit = np.array([color_hsv[0] + tolerance, 255, 255])
    
    return lowerlimit, upperlimit

In [16]:
yellow = [0, 255, 255]  # Yellow in RGB

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to HSV
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get the HSV limits for yellow color
    lowerlimit, upperlimit = get_limits(yellow)

    # Create a mask for yellow color
    mask = cv2.inRange(hsv_img, lowerlimit, upperlimit)
    mask_ = Image.fromarray(mask)
    bbox= mask_.getbbox()

    if bbox is not None:
        x1, y1, x2, y2 = bbox
        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)


    # Bitwise AND mask and original image to highlight detected color
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # Display the original frame and the result
    cv2.imshow("frame", frame)
    cv2.imshow("Result", result)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()